In [2]:
import pandas as pd
import numpy as np
import ddf_utils.ddf_reader as ddf
import xlrd
import os
import re

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
ddf.SEARCH_PATH = "/Users/semio/src/work/Gapminder/"

In [3]:
def to_concept_id_2(s, sub='[/ -\.\*–";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    if s is np.nan:
        return s

    s1 = re.sub(sub, sep, s.strip())
    s1 = s1.replace('\n', '')

    # remove the first/last underscore
    if s1[-1] == sep:
        s1 = s1[:-1]
    if s1[0] == sep:
        s1 = s1[1:]

    return s1.lower()


In [4]:
sheets = xlrd.open_workbook('source/bp-statistical-review-of-world-energy-2015-workbook.xlsx').sheet_names()

In [6]:
list(map(to_concept_id_2, sheets))

['contents',
 'oil_proved_reserves',
 'oil_proved_reserves_history',
 'oil_production_barrels',
 'oil_production_tonnes',
 'oil_consumption_barrels',
 'oil_consumption_tonnes',
 'oil_regional_consumption',
 'oil_spot_crude_prices',
 'oil_crude_prices_since_1861',
 'oil_refinery_capacities',
 'oil_refinery_throughputs',
 'oil_regional_refining_margins',
 'oil_trade_movements',
 'oil_inter_area_movements',
 'oil_imports_and_exports',
 'gas_proved_reserves',
 'gas_proved_reserves_history',
 'gas_production_bcm',
 'gas_production_bcf',
 'gas_production_tonnes',
 'gas_consumption_bcm',
 'gas_consumption_bcf',
 'gas_consumption_tonnes',
 'gas_trade_pipeline',
 'gas_trade_movements_lng',
 'gas_trade_2013_2014',
 'gas_prices',
 'coal_reserves',
 'coal_prices',
 'coal_production_tonnes',
 'coal_production_mtoe',
 'coal_consumption_mtoe',
 'nuclear_consumption_twh',
 'nuclear_consumption_mtoe',
 'hydro_consumption_twh',
 'hydro_consumption_mtoe',
 'other_renewables_twh',
 'other_renewables_mtoe'

In [40]:
data = pd.read_excel('source/bp-statistical-review-of-world-energy-2015-workbook.xlsx', na_values=['n/a'], 
              sheetname='Oil - Proved reserves history', 
              skiprows=2)

In [41]:
data = data.set_index('Thousand million barrels')

In [43]:
data = data[:'Total World']   ## should I also keep OECD related entry? current will drop them

In [44]:
data = data.dropna(how='all')

In [45]:
data = data.reset_index()

In [5]:
c = ddf.ddf_entities('ddf--gapminder--systema_globalis')['country']

In [6]:
cgw = c

def gw_id(s):
    for i in ['name', 'alternative_1', 'alternative_2', 'alternative_3', 'upper_case_name',
       'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6']:
    
        if s.lower() in cgw[i].str.lower().values:
            return cgw[cgw[i].str.lower() == s.lower()]['country'].values[0]
    return np.nan

In [46]:
data = data.rename(columns={'Thousand million barrels': 'geo_name'})
data['geo'] = data['geo_name'].map(to_concept_id_2)

In [47]:
# entities

data[['geo', 'geo_name']].to_csv('../ddf--entities--geo.csv', index=False)

In [48]:
data = data.drop(['2013.1', 'of total', 'geo_name'], axis=1)

In [50]:
data = data.set_index('geo')

In [51]:
data

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
geo,,,,,,,,,,,,,,,,,,,,,
us,36.533000,36.494000,35.079000,35.636001,36.088999,36.360000,35.054000,35.403001,35.063001,34.270000,...,29.922000,29.444000,30.459999,28.396000,30.871999,34.990000,39.775001,44.179999,48.462998,48.462998
canada,39.527521,40.175334,40.312097,40.491261,40.531767,40.939039,41.102001,41.201262,41.467760,41.295702,...,180.044115,179.361955,178.834031,176.349920,175.339445,174.904200,174.160624,173.557101,172.917313,172.917313
mexico,47.223999,56.998001,56.998001,57.096001,56.410000,55.592999,54.880001,54.110001,53.012001,51.983002,...,13.669700,12.849600,12.186600,11.865500,11.865500,11.691300,11.393700,11.423900,11.078800,11.078800
total_north_america,123.284521,133.667336,132.389098,133.223262,133.030766,132.892037,131.036002,130.714264,129.542761,127.548704,...,223.635815,221.655555,221.480630,216.611420,218.076944,221.585500,225.329325,229.161001,232.459111,232.459111
argentina,2.457000,2.171652,1.949600,2.449769,2.349000,2.240000,2.233000,2.246000,2.279900,2.167700,...,2.180263,2.586760,2.616020,2.520308,2.511496,2.524152,2.525000,2.354209,2.329584,2.327230
brazil,1.317967,1.475841,1.718439,1.849833,2.016010,2.168060,2.358415,2.550757,2.816137,2.759837,...,11.772262,12.182000,12.623830,12.801420,12.875670,14.246330,15.049890,15.314221,15.592741,16.154446
colombia,0.550000,0.530000,0.610000,0.640000,1.100000,1.240000,1.700000,1.910000,2.050000,1.980000,...,1.453200,1.509400,1.510000,1.362000,1.360000,1.900000,1.987600,2.200000,2.377000,2.445000
ecuador,0.973900,0.904100,0.914000,0.881700,1.137100,1.148000,1.235100,1.594300,1.514900,1.441600,...,4.865830,4.464930,4.001000,6.518000,6.332980,6.157800,7.210000,8.383030,8.190940,7.999560
peru,0.628445,0.882037,0.775145,0.696324,0.670446,0.589000,0.535800,0.480400,0.853000,0.847000,...,1.078258,1.097288,1.121480,1.121480,1.121480,1.240000,1.206200,1.422700,1.616952,1.616952


In [66]:
d = data.T.unstack()

In [67]:
d = d.dropna()

In [68]:
d = d.reset_index()
d.columns = ['geo', 'year', 'oil_proved_reserves_history']

In [69]:
d.sort_values(by=['geo', 'year']).to_csv('../ddf--datapoints--oil_proved_reserves_history--by--geo--year.csv',
                                         index=False)

In [7]:
def extract(sheet_name, ddf_id):
    data = pd.read_excel('source/bp-statistical-review-of-world-energy-2015-workbook.xlsx', na_values=['n/a'], 
              sheetname=sheet_name, 
              skiprows=2)
    
    data = data.rename(columns={data.columns[0]: 'geo'})
    data['geo'] = data['geo'].map(to_concept_id_2)
    
    data = data.set_index('geo')
    data = data.dropna(how='all')
    
    data = data[:'total_world']
    
    data = data.reset_index()

    data = data.drop(['2013.1', 'of total'], axis=1)
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]
    
    return d.sort_values(by=['geo', 'year'])

In [102]:
sss = 'Electricity Generation '
print(ddd[sss])
df_2 = extract(sss, ddd[sss])

electricity_generation


In [103]:
fn = os.path.join('..', 'ddf--datapoints--'+ddd[sss]+'--by--geo--year.csv')

df_2.to_csv(fn, index=False)

In [12]:
ddd = dict(zip(sheets, list(map(to_concept_id_2, sheets))))

### Notes:

pay attention to the one failed below.

In [14]:
for i in ddd.keys():
    try:
        df_2 = extract(i, ddd[i])
    except:
        print(i, 'failed')
        continue
    print(ddd[i], '...')
    fn = os.path.join('..', 'ddf--datapoints--'+ddd[i]+'--by--geo--year.csv')

    df_2.to_csv(fn, index=False)

Gas - Trade - pipeline failed
other_renewables_twh ...
Oil –  Spot crude prices failed
gas_proved_reserves_history ...
Oil - Imports and exports failed
electricity_generation ...
coal_consumption_mtoe ...
Primary Energy - Cons by fuel failed
geo_biomass_other_twh ...
Approximate conversion factors failed
Oil - Trade movements failed
gas_production_bcf ...
oil_production_tonnes ...
biofuels_production_ktoe ...
oil_refinery_throughputs ...
hydro_consumption_mtoe ...
gas_consumption_bcf ...
wind_consumption_twh ...
Geothermal capacity failed
Coal - Reserves failed
coal_production_tonnes ...
carbon_dioxide_emissions ...
oil_proved_reserves_history ...
nuclear_consumption_mtoe ...
gas_production_tonnes ...
coal_production_mtoe ...
oil_production_barrels ...
Definitions failed
hydro_consumption_twh ...
gas_consumption_bcm ...
Gas – Proved reserves failed
primary_energy_consumption ...
Wind capacity failed
solar_consumption_twh ...
Coal - Prices failed
Oil – Proved reserves failed
Oil - Crude

In [15]:
c1 = []

for i in ddd.keys():

    fn = os.path.join('..', 'ddf--dataponts--'+ddd[i]+'--by--geo--year.csv')
    
    if os.path.exists(fn):
        c1.append(ddd[i])

In [20]:
c2 = []

for i in ddd.keys():

    fn = os.path.join('..', 'ddf--dataponts--'+ddd[i]+'--by--geo--year.csv')
    
    if os.path.exists(fn):
        c2.append(i)

In [22]:
c1

['other_renewables_twh',
 'gas_proved_reserves_history',
 'electricity_generation',
 'coal_consumption_mtoe',
 'geo_biomass_other_twh',
 'gas_production_bcf',
 'oil_production_tonnes',
 'biofuels_production_ktoe',
 'oil_refinery_throughputs',
 'hydro_consumption_mtoe',
 'gas_consumption_bcf',
 'wind_consumption_twh',
 'coal_production_tonnes',
 'carbon_dioxide_emissions',
 'oil_proved_reserves_history',
 'nuclear_consumption_mtoe',
 'gas_production_tonnes',
 'coal_production_mtoe',
 'oil_production_barrels',
 'hydro_consumption_twh',
 'gas_consumption_bcm',
 'primary_energy_consumption',
 'solar_consumption_twh',
 'other_renewables_mtoe',
 'gas_consumption_tonnes',
 'oil_consumption_barrels',
 'biofuels_production_kboed',
 'oil_consumption_tonnes',
 'nuclear_consumption_twh',
 'gas_production_bcm',
 'geo_biomass_other_mtoe',
 'wind_consumption_mtoe',
 'solar_consumption_mtoe',
 'oil_refinery_capacities',
 'geo',
 'geo_name']

In [17]:
df_2.columns

Index(['geo', 'year', 'oil_refinery_capacities'], dtype='object')

In [18]:
c1.append('geo')
c1.append('geo_name')

In [23]:
c2.append('Geo')
c2.append('Geo Name')

In [24]:
concepts = pd.DataFrame([], columns=['concept', 'concept_type', 'name'])

In [25]:
concepts['concept'] = c1
concepts['name'] = c2

In [27]:
concepts['concept_type'] = 'measure'

In [28]:
concepts

,concept,concept_type,name
0,other_renewables_twh,measure,Other renewables -TWh
1,gas_proved_reserves_history,measure,Gas - Proved reserves history
2,electricity_generation,measure,Electricity Generation
3,coal_consumption_mtoe,measure,Coal Consumption - Mtoe
4,geo_biomass_other_twh,measure,Geo Biomass Other - TWh
5,gas_production_bcf,measure,Gas Production – Bcf
6,oil_production_tonnes,measure,Oil Production – Tonnes
7,biofuels_production_ktoe,measure,Biofuels Production - Ktoe
8,oil_refinery_throughputs,measure,Oil - Refinery throughputs
9,hydro_consumption_mtoe,measure,Hydro Consumption - Mtoe


In [29]:
concepts.loc[34, 'concept_type'] = 'entity_set'

In [30]:
concepts.loc[35, 'concept_type'] = 'string'

In [33]:
concepts = concepts.append(pd.DataFrame([['name', 'string', 'Name']], index=[36], columns=concepts.columns))

In [34]:
concepts.to_csv('../ddf--concepts.csv', index=False)

In [35]:
from ddf_utils.index import create_index_file

In [38]:
create_index_file('../')

,key,value,file
0,concept,concept_type,ddf--concepts.csv
1,concept,name,ddf--concepts.csv
0,"geo,year",biofuels_production_kboed,ddf--datapoints--biofuels_production_kboed--by...
0,"geo,year",biofuels_production_ktoe,ddf--datapoints--biofuels_production_ktoe--by-...
0,"geo,year",carbon_dioxide_emissions,ddf--datapoints--carbon_dioxide_emissions--by-...
0,"geo,year",coal_consumption_mtoe,ddf--datapoints--coal_consumption_mtoe--by--ge...
0,"geo,year",coal_production_mtoe,ddf--datapoints--coal_production_mtoe--by--geo...
0,"geo,year",coal_production_tonnes,ddf--datapoints--coal_production_tonnes--by--g...
0,"geo,year",electricity_generation,ddf--datapoints--electricity_generation--by--g...
0,"geo,year",gas_consumption_bcf,ddf--datapoints--gas_consumption_bcf--by--geo-...


In [40]:
for c in c2:
    print(c)

Other renewables -TWh
Gas - Proved reserves history 
Electricity Generation 
Coal Consumption -  Mtoe
Geo Biomass Other - TWh
Gas Production – Bcf
Oil Production – Tonnes
Biofuels Production - Ktoe
Oil - Refinery throughputs
 Hydro Consumption - Mtoe
Gas Consumption – Bcf
Wind Consumption - TWh 
Coal Production - Tonnes
Carbon Dioxide Emissions
Oil - Proved reserves history
Nuclear Consumption - Mtoe
Gas Production – tonnes
 Coal Production - Mtoe
Oil Production – Barrels
Hydro Consumption - TWh
Gas Consumption – Bcm
Primary Energy Consumption 
Solar Consumption - TWh
Other renewables - Mtoe
Gas Consumption – tonnes
Oil Consumption – Barrels
Biofuels Production - Kboed
Oil Consumption – Tonnes
Nuclear Consumption - TWh
Gas Production – Bcm
Geo Biomass Other - Mtoe
Wind Consumption - Mtoe
Solar Consumption - Mtoe
Oil - Refinery capacities
Geo
Geo Name


* Oil - Proved reserves history
* Oil Production – Barrels
* Oil Production – Tonnes
* Oil Consumption – Barrels
* Oil Consumption – Tonnes
* Oil - Refinery capacities
* Oil - Refinery throughputs
* Gas - Proved reserves history
* Gas Production – Bcm
* Gas Production – Bcf
* Gas Production – tonnes
* Gas Consumption – Bcm
* Gas Consumption – Bcf
* Gas Consumption – tonnes
* Coal Production - Tonnes
*  Coal Production - Mtoe
* Coal Consumption -  Mtoe
* Nuclear Consumption - TWh
* Nuclear Consumption - Mtoe
* Hydro Consumption - TWh
*  Hydro Consumption - Mtoe
* Other renewables -TWh
* Other renewables - Mtoe
* Solar Consumption - TWh
* Solar Consumption - Mtoe
* Wind Consumption - TWh
* Wind Consumption - Mtoe
* Geo Biomass Other - TWh
* Geo Biomass Other - Mtoe
* Biofuels Production - Kboed
* Biofuels Production - Ktoe
* Primary Energy Consumption
* Electricity Generation
* Carbon Dioxide Emissions

In [36]:
a1 = ddf.ddf_datapoint('ddf--gapminder--gapminder_world', 'oil_proved_reserves_total').query(
    "geo == 'usa'"
)

In [35]:
a2 = ddf.ddf_datapoint('ddf--bp--energy', 'oil_proved_reserves_history').query(
    "geo == 'us'"
)

In [41]:
a1.set_index('time')['oil_proved_reserves_total'] / (a2.set_index('year')['oil_proved_reserves_history'] * 136400000)

1980    0.999980
1981    1.000044
1982    1.000047
1983    1.000051
1984    1.000094
1985    1.000100
1986    0.999924
1987    1.000006
1988    1.000085
1989    0.999908
1990    1.000049
1991    1.000065
1992    0.999968
1993    0.999899
1994    0.999970
1995    1.000025
1996    0.999957
1997    1.000050
1998    0.999920
1999    0.999969
2000    0.999953
2001    1.000029
2002    1.000114
2003    0.999915
2004    0.999904
2005    0.999912
2006    0.999960
2007    1.000062
2008    1.072752
2009         NaN
2010         NaN
2011         NaN
2012         NaN
2013         NaN
2014         NaN
dtype: float64